# Chapter 6. User and Authentication

## 6.1 Introduction

### 6.1.1 Authentication with `Flask-Login`

(1) Configuration

(2) Adapting the `User` Class for login

### 6.1.2 Securing views

Using the `@login_required` decorator

### 6.1.3 Adding a login view, form and template

Add don't forget logout

### 6.1.4 Adding signup for new users

## 6.2 Demo: `Flask-Login` setup

Install the Flask extension package `flask-login`:

```bash
$ pip install flask-login
```

In [ ]:
# SAVE AS __init__.py

# -*- coding: utf-8 -*-

import os

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager 

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
app.config['DEBUG'] = True

db = SQLAlchemy(app)

# Configure authentication
login_manager = LoginManager()
login_manager.session_protection = 'strong'
login_manager.init_app(app)

from . import models
from . import views

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for
from flask_login import login_required

from . import app, db
from thermos.forms import BookmarkForm
from thermos.models import User, Bookmark 
    
# Fake login
def logged_in_user():
    return User.query.filter_by(username='reindert').first()

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=logged_in_user(), url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

## 6.3 Demo: Preparing the `User` model

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin

from . import db

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    
    def __repr__(self):
        return '<User %r>' % self.username

## 6.4 Demo: Adding the login page

Create a new template `login.html`.

In [ ]:
# SAVE AS forms.py

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField, PasswordField, BooleanField, SubmitField
from flask.ext.wtf.html5 import URLField
from wtforms.validators import DataRequired, url

class BookmarkForm(Form):
    url = URLField('The URL for your bookmark', validators=[DataRequired(), url()])
    description = StringField('Add an optional description:')
    
    def validate(self):
        if not (self.url.data.startswith("http://") or\
            self.url.data.startswith("https://")):
            self.url.data = "http://" + self.url.data
            
        if not Form.validate(self):
            return False
            
        if not self.description.data:
            self.description.data = self.url.data
            
        return True
        
class LoginForm(Form):
    username = StringField('Your Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Keep me logged in')
    submit = SubmitField('Log In')

In [ ]:
# SAVE As views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request
from flask_login import login_required, login_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm
from thermos.models import User, Bookmark 
    
# Fake login
def logged_in_user():
    return User.query.filter_by(username='reindert').first()
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=logged_in_user(), url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.query.filter_by(username=form.username.data).first()
        if user is not None:
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('index'))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

## 6.5 Demo: Redirect after login

When an anonymous user is trying to access some page which requires login, he/she will be directed to the login page and will then be redirected to the `next` page if he/she successfully logs in.

```python
return redirect(request.args.get('next') or url_for('index'))
```

In [ ]:
# SAVE AS __init__.py

# -*- coding: utf-8 -*-

import os

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager 

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
app.config['DEBUG'] = True

db = SQLAlchemy(app)

# Configure authentication
login_manager = LoginManager()
login_manager.session_protection = 'strong'
login_manager.login_view = 'login'
login_manager.init_app(app)

from . import models
from . import views

## 6.6 Review: `Flask-Login` overview

### 6.6.1 Setting up `flask-login`

(1) Install

```bash
$ pip install flask-login
```

(2) Import and configure

```python
from flask_login import LoginManager 
login_manager = LoginManager()
login_manager.session_protection = 'strong'
login_manager.login_view = 'login'
login_manager.init_app(app)
```

### 6.6.2 Setting up the `User` class

(1) Declare a `user_loader`:

* Tells `flask-login` how to retrieve a user by `id`.
* `id` is stored in the HTTP session.

```python
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))
```

(2) Inherit from UserMixin:

Default implementations for `is_authenticated`, `get_id`, etc

```python
class User(db.Model, UserMixin):
```

### 6.6.3 Using `flask-login`

(1) Mark views with `@login_required`

(2) The `current_user` variable holds the currently logged in user

(3) Log a user in with `login_user(user)`

Optional argument: `remember(bool)`

(4) Log them out with `logout_user()`.

(5) In a view:

`{% if current_user.is_authenticated() %}`

## 6.7 Demo: Current_user and logout

Note that `is_authenticated` is an attribute of `current_user` not a method.

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm
from thermos.models import User, Bookmark 
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=current_user, url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.query.filter_by(username=form.username.data).first()
        if user is not None:
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('index'))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

## 6.8 Demo: Password hashing

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin
from werkzeug.security import check_password_hash, generate_password_hash

from . import db

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    password_hash = db.Column(db.String)
    
    @property
    def password(self):
        raise AttributeError('password: write-only field')
        
    @password.setter
    def password(self, password):
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
        
    @staticmethod
    def get_by_username(username):
        return User.query.filter_by(username=username).first()
    
    def __repr__(self):
        return '<User %r>' % self.username

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm
from thermos.models import User, Bookmark 
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=current_user, url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.get_by_username(form.username.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('user', username=user.username))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

In [ ]:
# SAVE AS manager.py
# Create some initial users with password for testing without a signup page.

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from flask.ext.script import Manager, prompt_bool

from thermos import app, db
from thermos.models import User

manager = Manager(app)

@manager.command
def initdb():
    db.create_all()
    db.session.add(User(username='reindert', email='reindert@example.com', password='test'))
    db.session.add(User(username='arjen', email='arjen@example.com', password='test'))
    db.session.commit()
    print('Initialized the database')
    
@manager.command
def dropdb():
    if prompt_bool(
        'Are you sure you want to lose all your data'):
        db.drop_all()
        print('Dropped the database')
        
if __name__ == '__main__':
    manager.run()

## 6.9 Demo: Adding a signup page

Create a new template page `signup.html`.

In [ ]:
# SAVE AS forms.py

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField, PasswordField, BooleanField, SubmitField
from flask.ext.wtf.html5 import URLField
from wtforms.validators import DataRequired, Length, Email, Regexp, EqualTo,\
    url, ValidationError
    
from thermos.models import User

class BookmarkForm(Form):
    url = URLField('The URL for your bookmark', validators=[DataRequired(), url()])
    description = StringField('Add an optional description:')
    
    def validate(self):
        if not (self.url.data.startswith("http://") or\
            self.url.data.startswith("https://")):
            self.url.data = "http://" + self.url.data
            
        if not Form.validate(self):
            return False
            
        if not self.description.data:
            self.description.data = self.url.data
            
        return True
        
class LoginForm(Form):
    username = StringField('Your Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Keep me logged in')
    submit = SubmitField('Log In')
    
class SignupForm(Form):
    username = StringField('Username',
                    validators=[
                        DataRequired(), Length(3, 80),
                        Regexp('^[A-Za-z0-9_]{3,}$',
                            message='Usernames consist of numbers, letters, '
                                    'and underscores.')])
    password = PasswordField('Password',
                    validators=[
                        DataRequired(),
                        EqualTo('password2', message='Passwords must match')])
    password2 = PasswordField('Confirm password', validators=[DataRequired()])
    email = StringField('Email',
                        validators=[DataRequired(), Length(1, 80), Email()])
                        
    def validate_email(self, email_field):
        if User.query.filter_by(email=email_field.data).first():
            raise ValidationError('There already is a user with this email address.')
            
    def validate_username(self, username_field):
        if User.query.filter_by(username=username_field.data).first():
            raise ValidationError('This username is already taken.')

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin
from werkzeug.security import check_password_hash, generate_password_hash

from . import db

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    password_hash = db.Column(db.String)
    
    @property
    def password(self):
        raise AttributeError('password: write-only field')
        
    @password.setter
    def password(self, password):
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
        
    @staticmethod
    def get_by_username(username):
        return User.query.filter_by(username=username).first()
    
    def __repr__(self):
        return '<User %r>' % self.username

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm, SignupForm
from thermos.models import User, Bookmark 
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=current_user, url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.get_by_username(form.username.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('user', username=user.username))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))
    
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    form = SignupForm()
    if form.validate_on_submit():
        user = User(email=form.email.data,
                    username=form.username.data,
                    password=form.password.data)
        db.session.add(user)
        db.session.commit()
        flash('Welcome, {}! Please login'.format(user.username))
        return redirect(url_for('login'))
    return render_template('signup.html', form=form)
        
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

## 6.10 Resources and summary

### 6.10.1 Resources

`flask-login`: http://flask-login.readthedocs.org/en/latest

### 6.10.2 Summary

(1) `flask-login`

* `user_loader`
* `UserMixin`
* `login_required`
* `current_user`
* `login_user()`
* `logout_user()`

(2) Password hashing

(3) Signup